# Closest N points

Given points in a cartesian plane (x,y), n points in that plane, and a point of interest (x_1, y_1), find the n closest points to the point of interest.

Define the points in an NxD array 

## Solution

Dynamic programming

Keep a sorted solution set.  For each new candidate point, test it against the furthers kept point.  If it's smaller, insert it in order.

## Complexity

We need to touch every point once, so that's at least O(N).  
Inserting into our solution set could cost O(m) to maintain order.
In the worst case, keeping a large number of the points, in a set of points with descending distance, we'd have to insert every new point, and maintain order in our solition set.

So the solution costs O(n X m), but in most cases the m portion is insignificant compared to n, so it would be O(n)

We also need to keep our solution set in memory, but no more, so the space complexity is O(m)

In [3]:
import numpy as np

def L2_norm(a, b):
    # L2 norm
    return np.sqrt( (a[0]-b[0])**2 + (a[1]-b[1])**2 )
        
    
def find_m_closest_points(points_to_search, point_of_interest, m, dist_func = L2_norm):
    closest_points = []
    
    # collect the first closest points
    for pi_idx in range(m):
        pi = all_points_array[pi_idx]
        closest_points.append([dist_func(point_of_interest, pi), pi_idx])

    closest_points = sorted(closest_points)
    
    # test all the rest
    for idx, pi in enumerate(all_points_array[m:]):
        delta_i = dist_func(point_of_interest, pi)
        # keeps most recent for ties
        if delta_i <= closest_points[-1][0]:
            # drop further currently kept
            closest_points.pop(-1)
            new_point_tup = [delta_i, idx]
            inserted = False
            for jdx, kept_point in enumerate(closest_points):
                if delta_i <= kept_point[0]:
                    closest_points = closest_points[:jdx] + [new_point_tup] + closest_points[jdx:]
                    inserted = True
                    break
            # if we haven't placed yet, it goes at the end
            if not inserted:
                closest_points.append(new_point_tup)
    
    return closest_points



n = 1000
m = 5
points_range = 10
point_of_interest = np.array([1.0,1.0])
all_points_array = (np.random.rand(n,2) * points_range) - (points_range/2)

m_closest_points = find_m_closest_points(all_points_array, point_of_interest, m, L2_norm)

print("{0} closest points to {1} are :".format(m, point_of_interest))
print("")
for delta_i, idx in m_closest_points:
    s = "{0}, at a distance of : {1}".format(all_points_array[idx], delta_i)
    print(s)


5 closest points to [ 1.  1.] are :

[ 2.40378091 -3.86121024], at a distance of : 0.06282852869857627
[ 1.09309992  2.93149129], at a distance of : 0.31370970116522423
[-4.37000299 -1.15437599], at a distance of : 0.3660390362216514
[-3.08887716 -0.26091244], at a distance of : 0.37210137381041425
[ 2.44623445 -4.93592745], at a distance of : 0.4872902483369646


# Find the largest Square in an NxM matrix with obstructions

Given an NxM matrix, with zeros and ones (open spaces and obstructions), find the largest sub-matrix, that is square dxd (including 1x1), with no obstructions within it.

## Solution:

Using dynamic programming and memoization we can fill a memoization matrix, of the same size as the input, with previously computed solutions.  

The solution at x, y is the minimum of the previously computed solutions at: 

x+1, y  

x, y+1

x+1, y+1

## Complexity:

We have to touch every element of the input matrix once, so the run-time is O(n X m)

We have to keep a memoization matrix of the same size, so our space complexity is O(n X m)

In [6]:
# generate matrix for problem
from random import shuffle
def create_nxm_matrix_with_obstructions(n, m, n_obs):
    """ 
    n, m are size of matrix
    n_obs is the number of osbtructions
    """
    mat = np.zeros([n,m])
    indices = [(i,j) for i in range(n) for j in range(m)]
    shuffle(indices)
    for i,j in indices[:n_obs]:
        mat[i][j] = 1
        
    return mat


def find_largest_unobstructed_square(mat):
    n, m  = mat.shape
    mem_mat = np.ones([n,m]) * -1
    maxs = 0
    maxs_x = 0
    maxs_y = 0
    
    def solve(x, y):
        nonlocal n, m, mat, mem_mat, maxs, maxs_x, maxs_y

        if y > n - 1 or x > m - 1:
            return 0
        
        if not mem_mat[y][x] == -1:
            # we've computed this before, so use memo
            return mem_mat[y][x]
        
        # base case
        if y - 1 == n or y - 1 == m:
            # we hit the edge
            if mat[y][x] == 1:
                res = 0
            else:
                res = 1
                
        elif mat[y][x] == 0:
            # recurse three neighbors, take min of those solutions
            s1 = solve(x+1, y)
            s2 = solve(x, y+1)
            s3 = solve(x+1, y+1)
            res = min(s1, s2, s3)
            if res > 0:
                if y + res < n - 1 and x + res < m - 1:
                    res += 1
            else:
                # base case
                res = 1
        else:
            # recurse three neighbors
            solve(x+1, y)
            solve(x, y+1)
            solve(x+1, y+1)
            res = 0
        
        if res > maxs:
            maxs = res
            maxs_x = x
            maxs_y = y
        
        mem_mat[y][x] = res
        
        return res
    
    
    solve(0, 0)
    print("For input matrix:")
    print(mat)
    print("")
    print("Largest square has size of {0}, starting at x = {1} and y = {2}".format(maxs, maxs_x, maxs_y))
    print("")
    print("The Memo. matrix looks like: ")
    print(mem_mat)

In [7]:
n = 10
m = 8
n_obs = 8
mat = create_nxm_matrix_with_obstructions(n, m, n_obs)

find_largest_unobstructed_square(mat)

For input matrix:
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]]

Largest square has size of 4.0, starting at x = 1 and y = 1

The Memo. matrix looks like: 
[[ 2.  4.  3.  2.  2.  2.  1.  1.]
 [ 1.  4.  3.  2.  1.  1.  1.  1.]
 [ 0.  3.  3.  2.  1.  0.  0.  1.]
 [ 2.  2.  3.  3.  2.  2.  1.  1.]
 [ 1.  1.  2.  2.  2.  1.  1.  1.]
 [ 1.  0.  2.  1.  1.  1.  0.  1.]
 [ 3.  3.  2.  1.  0.  2.  1.  1.]
 [ 2.  2.  2.  2.  2.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  0.]]


# Given distances, find positions of points

There are N Gas stations and we are given the distance between each gas station.

So we have nC2 distance.

Find the positions of all gas stations given the distances.

## Solution

This depends on how our distances are represented, and how we are representing our world.
If we're assuming we're in a plane, with x,y pair positions, and our distances are in i_delta_j_x, i_delta_j_y) pairs, we can solve this arbitrarily by setting one of the gas stations, station i, to position (0,0), and do a single pass through the distance matrix to compute the locations from station j with (0+i_delta_j_x, 0+i_delta_j_y).

# Smallest Pairs in sorted arrays

Given two sorted arrays, A and B, find the first k pairs (a,b) from A and B, which have the smallest sum.

E.g.

A = [1,2,3,6,10]
B = [1,4,5,7]
k = 5
res = [(1,1),(1,4),(1,5),(2,1),(3,1)]

## Solution

Pointers and walking

One pointer for each of:

Next time we use (a,b), the index in A for a, and the index in B for b
Next time we use (b,a), the index in B for b, and the index in A for a

We run in a while loop until we've pupulated the output with k pairs, and do comparisons of (a,b) and (b,a) to choose the smallest.  The trickiest part is updating the pointers correctly based on the cases.

## Complexity

We're doing a constant amount of comparisons and lookups, by index O(1), for each time we select the next output, so we're bounded above by the number of pairs we want.  So run-time is O(k).

Space complexity is trivial, a pair of pointers with two ints, and an output list of size K, so O(k) again.

In [16]:
from random import randint

def gen_arrays(len_a,len_b):
    A = sorted([randint(1,10) for i in range(len_a)])
    B = sorted([randint(1,10) for i in range(len_b)])
    return (A, B)


def find_smallest_pairs(A, B, k):
    # defective lists
    if len(A) == 0 or len(B) == 0:
        return False
    # less pairs than k
    if len(A) * len(B) <= k:
        return [(a,b) for a in A for b in B]
    
    # first pair always in solution
    res = [(A[0],B[0])] 
    
    A_next = [0, 1]
    B_next = [0, 1]
    
    while len(res) < k:
        if A[A_next[0]] > len(A) - 1:
            # if A expended, finish with B
            while len(res) < k:
                res.append( (B[B_next[0]], A[B_next[1]]) )
                B_next[1] += 1
        elif B[B_next[0]] > len(B) - 1:
            # if B expended, finish with A
            while len(res) < k:
                res.append( (A[A_next[0]], B[A_next[1]]) )
                A_next[1] += 1
        else:
            a_candidate = (A[A_next[0]], B[A_next[1]])
            b_candidate = (B[B_next[0]], A[B_next[1]])
            if sum(a_candidate) <= sum(b_candidate):
                res.append(a_candidate)
                A_next[1] += 1
                if A_next[1] > len(B) - 1:
                    A_next[0] += 1
                    A_next[1] = B_next[0] + 1
            else:
                res.append(b_candidate)
                B_next[1] += 1
                # if B next right has hit the end, move B next up one
                # B next right is set to A left + 1
                if B_next[1] > len(A) - 1:
                    B_next[0] += 1
                    B_next[1] = A_next[0] + 1
                 
    return res


len_a = 5
len_b = 4
k = 5
A, B = gen_arrays(len_a, len_b)
print("A list is : {}".format(A))
print("B list is : {}".format(B))
print("Smallest {} sum pairs are :".format(k))
find_smallest_pairs(A, B, k)

A list is : [5, 6, 6, 6, 8]
B list is : [5, 5, 6, 10]
Smallest 5 sum pairs are :


[(5, 5), (5, 6), (5, 6), (5, 6), (5, 8)]

# Path finding

Given a matrix of 0s and 1s, where 1s are walls, navigate from the top left to the bottom right in the shortest path.

More complex version gives costs to each space.

## Solution

A* is the most efficient graph traversal, path finding algorithm in most cases, with a run-time of O(E) and space complexity of O(V).

This is an extension of Djikstra's algorithm.

# Convert a binary tree to a list and back

In [17]:
#todo

# Given a boolean matrix to tree

A boolean matrix represents a tree.  If entry (i,j) is 1, i is the parent of j.

Construct this tree.

Tree should be a simple class, with only it's id, and pointers to it's children

Node:
    - id
    - left
    - right

## Solution

Preconstruct all of the nodes, with no pointers to thier childred, and store them in an random accessable array.

Walk the rows of the matrix, i, walk the row, and for each 1 at i,j in the row, set one of the children of i to j.

This doesn't guarentee rights and lefts in the tree, but that information was lost in the matrix representation anyway.

We need to return the root node at the end, so we need to find it somehow.

We can do that during our single pass by coutning up how many parents every node has.  The one with 0 parents is the root. O(n) to find it at the ned, but that is less than the core complexity.

## Complexity

There is no way to avoid looking at all the entries in the matrix, so at best we could do O(n^2).  This solution achieves that.

We need O(n) to create the nodes, and we need a container for them.  With a well contructed container, using pointers, this trivial.


We can generalize this to a non-binary tree by setting the children field to a map

In [2]:
class Node:
    
    def __init__(self, node_id, children = {}):
        self._id = node_id
        self._children = children
    
    def add_child(self, child):
        self._children[child._id] = child
        

def construct_tree(mat):
    # O(n)
    nodes = [Node(i) for i in range(len(mat))]
    no_parents = set(range(len(mat)))
                                    
    # O(n**2)
    for parent_id, is_parent_row in enumerate(mat):
        for child_id, is_parent in enumerate(is_parent_row):
            if is_parent:
                no_parents.remove(child_id)
                nodes[parent_id].add_child(nodes[child_id])
    
    return [nodes[i] for i in no_parents]


"""
          4
       0    1
      2 3  x  5
    x x x x  x x
    
"""

mat = [
    [0,0,1,1,0,0],
    [0,0,0,0,0,1],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [1,1,0,0,0,0],
    [0,0,0,0,0,0]
]

root = construct_tree(mat)

# Find subsets that have property Min(subset) + Max(subset) < k

Given a sorted array, find all the subsets, S_i, within the array which contain the property Min(S_i) + Max(S_i) = k, for some value k.


## Solution
With two pointers at either end of the array, walk the right (end) pointer from the end of the array towards the beginning until you find a[0] + a[i] < k.  This is your first solution.  All subsets from  from a[0] to a[i] should be added to the solition set.  Then step the left (beginning), up one position.  Check again, and if it doesn't meet the condition, walk the right pointer leftwards until you find the next solution.  Rinse, and repeate until the right pointer passes the left pointer.  Make sure to have them pass eachother, since a[i] + a[i]

## Complexity
This process has the pointers pass over the array once, so O(n), linear, time.  However, creating the output sets can be expensive.  If we have a solution from index 3 to 9, we have to produce 7 sets of numbers at indeces [3], [3,4], [3,4,5], and so on.  This is a pretty expensive operation if our array is large and our k is large.  For the case of k > arr[0] and arr[n-1], our output is N! in size. We can save a lot of space of time if we just output the pairs of min_idx, max_idx where all subsets are solitions.  This get's us back to O(N)

Space complexity can be pretty big here since the output set can be O(n!).  We can save on space complexity if we just output the pairs of min_idx, max_idx where all subsets are solitions.  This get's us back to O(n).

In [9]:
def find_subsets_with_min_max_k(arr, k):
    # for constant time lookup of indexes
    p_left = 0
    p_right = len(arr) - 1
    res = []
    
    while p_right >= p_left:
        if arr[p_left] + arr[p_right] < k:
                res.extend(
                    [arr[p_left:i+1] for i in range(p_left, p_right+1)]
                )
                p_left += 1
        else:
            p_right -= 1
    
    return res

def find_subsets_with_min_max_k_condensed_output(arr, k):
    p_left = 0
    p_right = len(arr) - 1
    res = []
    while p_right >= p_left:
        if arr[p_left] + arr[p_right] < k:
                res.append( (p_left, p_right) )
                p_left += 1
        else:
            p_right -= 1
    
    return res

arr = list(range(3,12))
k = 12
print(arr)
print("Full output : {0}".format(find_subsets_with_min_max_k(arr, k)))
print("Limits indices output : {0}".format(find_subsets_with_min_max_k_condensed_output(arr, k)))

[3, 4, 5, 6, 7, 8, 9, 10, 11]
Full output : [[3], [3, 4], [3, 4, 5], [3, 4, 5, 6], [3, 4, 5, 6, 7], [3, 4, 5, 6, 7, 8], [4], [4, 5], [4, 5, 6], [4, 5, 6, 7], [5], [5, 6]]
Limits indices output : [(0, 5), (1, 4), (2, 3)]
